In [1]:
import re
def preprocess(text):
    text = re.sub(r'<br \\>|\W',' ',text)
    words = text.split()
    
    return [w.lower() for w in words]

In [2]:
import numpy as np
import tensorflow as tf
import random
from collections import Counter
import os  
path = r"C:\Users\Me\Downloads\mlp project\aclImdb_v1\aclImdb\train\neg" #文件夹目录  
files= os.listdir(path) #得到文件夹下的所有文件名称  
maxlen = 0
documents = []
rates = []
vocab = []
for file in files: #遍历文件夹    
    f = open(path+"/"+file,encoding="utf-8"); #打开文件  
    iter_f = iter(f); #创建迭代器  
    rates.append(file[-5])
#     rates.append(file)
    for line in iter_f: #遍历文件，一行行遍历，读取文本
        words = preprocess(line)
        if len(words) > maxlen:
            maxlen = len(words)
        for word in words:
            if word not in vocab:
                vocab.append(word)
            
        documents.append(words)


In [3]:
path = r"C:\Users\Me\Downloads\mlp project\aclImdb_v1\aclImdb\train\pos" #文件夹目录  
files= os.listdir(path) #得到文件夹下的所有文件名称
for file in files: #遍历文件夹    
    f = open(path+"/"+file,encoding="utf-8"); #打开文件  
    iter_f = iter(f); #创建迭代器  
    if file[-5] == '0':
        rates.append(10)
    else:
        rates.append(file[-5])
#     rates.append(file)
    for line in iter_f: #遍历文件，一行行遍历，读取文本  
        words = preprocess(line)
        if len(words) > maxlen:
            maxlen = len(words)        
        for word in words:
            if word not in vocab:
                vocab.append(word)
            
        documents.append(words)


In [4]:
print(len(vocab))
with open('vocab.txt','w',encoding="utf-8") as f:
    for v in vocab:
        f.write(v)
        f.write('\n')

74891


In [12]:
vocab_to_int = {w: c for c, w in enumerate(vocab)}
int_to_vocab = {c: w for c, w in enumerate(vocab)}
int_documents = []
for document in documents:
#     int_document = [vocab_to_int[w] for w in document]
    int_document = []
    for w in document:
        if w in vocab:
            int_document.append(vocab_to_int[w])
    int_documents.append(np.array(int_document).astype(np.int))
print(int_documents[0])

[  221   222  2940 20864     5  6526    11     2   515     8   241   374
    70  9658  1670    52    31    17   176   375  1556     1   418   317
   286   488   327   225    46   988   234   947   332   228     1    31
   644   449     1   360   239  9823   116    31  8137    84 20864    35
   294   286  3344  1982    66    84   251   230   270   314   871    30
   418    40   225   988   234   947    31   294   581    54   547   947
   332    16     2   488  7000 49142 27137 27138    31  1503    16   253
   856    48    92   732     4  3560   253    35    69    70   569   509
  1024  1041    84  1148    41  3568     1     2 10460  8137   114   344
 19490   726    63    55  3238   191 20864  2139   279   581   286   489
   713    31  1386   942   680     1   209  7444 20864  2016   360   683
   332 27138    35  9823   581   286  1997   741 27138    16  2539    63
    55    31   245    12    41  4781 13924   281 15205    41  3526   335
    34   217   683   225   289   445    63   803   

In [13]:
int_documents = np.array(int_documents)
rates = np.array(rates).astype(np.int)
rates[rates<5] = 0
rates[rates>5] = 1

In [7]:
train_inputs = np.append(int_documents[:10000], int_documents[12500:22500])
train_targets = np.append(rates[:10000], rates[12500:22500])
valid_inputs = np.append(int_documents[10000:12500], int_documents[22500:])
valid_targets = np.append(rates[10000:12500], rates[22500:])

In [8]:
np.savez("train", inputs =train_inputs, targets =train_targets)
np.savez("valid", inputs =valid_inputs, targets =valid_targets)

In [9]:
f.close()

In [10]:
import numpy as np
import tensorflow as tf
import random
from collections import Counter
import os  
path = r"C:\Users\Me\Downloads\mlp project\aclImdb_v1\aclImdb\test\neg" #文件夹目录  
files= os.listdir(path) #得到文件夹下的所有文件名称  
maxlen = 0
documents = []
rates = []
for file in files: #遍历文件夹    
    f = open(path+"/"+file,encoding="utf-8"); #打开文件  
    iter_f = iter(f); #创建迭代器  
    rates.append(file[-5])
#     rates.append(file)
    for line in iter_f: #遍历文件，一行行遍历，读取文本
        words = preprocess(line)
        documents.append(words)


In [11]:
path = r"C:\Users\Me\Downloads\mlp project\aclImdb_v1\aclImdb\test\pos" #文件夹目录  
files= os.listdir(path) #得到文件夹下的所有文件名称
for file in files: #遍历文件夹    
    f = open(path+"/"+file,encoding="utf-8"); #打开文件  
    iter_f = iter(f); #创建迭代器  
    if file[-5] == '0':
        rates.append(10)
    else:
        rates.append(file[-5])
#     rates.append(file)
    for line in iter_f: #遍历文件，一行行遍历，读取文本  
        words = preprocess(line)
        documents.append(words)


In [14]:
np.savez("test", inputs =int_documents, targets =rates)

In [4]:
import numpy as np
train_data = np.load("train.npz")
valid_data = np.load("valid.npz")

In [62]:

print(len(train_data['targets']))
print(len(valid_data['inputs']))

20000
5000


In [15]:
import data_providers as data_providers
batch_size = 50
train_data = data_providers.ACLIMDBDataProvider("train",batch_size=batch_size)
valid_data = data_providers.ACLIMDBDataProvider("valid",batch_size=batch_size)


In [16]:
with open(r"C:\Users\Me\Downloads\mlp project\mlp\vocab.txt") as file_object:
    iter_f = iter(file_object)
    vocab = []
    for line in iter_f:
        vocab.append(line[:-1])


In [17]:
vocab_to_int = {w: c for c, w in enumerate(vocab)}
int_to_vocab = {c: w for c, w in enumerate(vocab)}


In [18]:
for c in train_data.next()[0][0]:
    print(int_to_vocab[c])

the
war
between
the
states
was
perhaps
the
darkest
hour
in
the
history
of
america
a
war
that
pitted
brother
against
brother
and
family
against
family
and
left
scars
that
even
today
have
not
yet
healed
and
in
all
probability
never
will
and
as
in
any
story
about
any
war
beyond
any
historical
significance
it
is
the
personal
discord
behind
the
greater
conflict
that
creates
the
emotional
impetus
that
makes
it
involving
it
is
the
human
element
that
renders
the
context
necessary
to
give
it
perspective
which
is
what
director
ang
lee
provides
in
ride
with
the
devil
a
civil
war
drama
in
which
he
focuses
on
the
personal
travails
within
the
broader
depiction
of
the
war
itself
and
along
the
way
manages
to
include
an
examination
of
one
of
the
bloodiest
chapters
of
the
war
the
infamous
raid
on
lawrence
kansas
by
quantrill
and
his
raiders
which
he
succeeds
in
presenting
quite
objectively
from
the
confederate
point
of
view
br
br
in
1863
the
union
influence
predominates
in
the
state
of
kansas
and
even
a